# Application

# Introduction

The main goal of this chapter is to clearly demonstrate, using an applied example, how to use EEG signals for applied problems.
Here we will look at an example of determining stress levels. And here we have the most interesting and complicated tasks.


# Import libraries
To install libraries in Python, you can use a package manager like pip, which comes pre-installed with most Python distributions.

In [1]:
import sys
import time
import  matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import pandas as pd
from scipy import signal
import numpy as np
from scipy.integrate import simps
from scipy import signal
import re

# Import Dataset
Here we have an example dataset of mornings and evenings after a busy day at work. Stress is expected to be low in the morning and high in the evening. This is a great dataset for detecting stress levels.

In [4]:
data_1_morning = "datasets/dataset_emotional/1_morning.xlsx"
data_1_evening = "datasets/dataset_emotional/1_evening.xlsx"
data_2_morning = "datasets/dataset_emotional/2_morning.xlsx"
data_2_evening = "datasets/dataset_emotional/2_evening.xlsx"
data_3_morning = "datasets/dataset_emotional/3_morning.xlsx"
data_3_evening = "datasets/dataset_emotional/3_evening.xlsx"
data_4_morning = "datasets/dataset_emotional/4_morning.xlsx"
data_4_evening = "datasets/dataset_emotional/4_evening.xlsx"
data_5_morning = "datasets/dataset_emotional/5_morning.xlsx"
data_5_evening = "datasets/dataset_emotional/5_evening.xlsx"

data = pd.read_excel(data_1_morning)
print(data.columns)
print(data.shape)
data = data["ch1"]
plt.xlabel('Samples')
plt.ylabel('EEG, μV')
plt.title('EEG sample')
plt.plot(data[2000:8000]) # use the clean data
plt.show()